In [143]:
import yaml
import glob
from collections import OrderedDict

import qiime2.plugins
import importlib
from qiime2.sdk import Result

In [57]:
final_artifact = Result.load('diff.qzv')
results = {}
for result in glob.glob('*.qz*'):
    results[Result.peek(result).uuid] = result

In [172]:
yaml.add_constructor('!ref', lambda x, y: y)
yaml.add_constructor('!cite', lambda x, y: y)

def get_import(action, prov_dir, results, uuid):
    cmd = ['qiime', 'tools', 'import', '--input-path']
    assert len(action['action']['manifest']) == 1
    cmd.append(action['action']['manifest'][0]['name'])
    cmd.append('--input-format')
    cmd.append(action['action']['format'])
    cmd.append('--type')
    with (prov_dir / 'artifacts' / uuid / 'metadata.yaml').open() as fh:
        metadata = yaml.load(fh)
    cmd.append(metadata['type'])
    cmd.append('--output-path')
    cmd.append(results[uuid])
    return ' '.join(cmd)
    
def get_command(action, results, prov_dir, uuid):
    if action['action']['type'] == 'import':
        return get_import(action, prov_dir, results, uuid), []
    cmd = ['qiime']
    cmd.append(action['action']['plugin'].value.split(':')[-1])
    cmd.append(action['action']['action'].replace('_', '-'))
    for param_dict in action['action']['parameters']:
        (param, value), = param_dict.items()
        mod = importlib.import_module(
            'qiime2.plugins.' + 
            action['action']['plugin'].value.split(':')[-1].replace('-', '_'),
        )
        parameters = getattr(mod.actions, action['action']['action']).signature.parameters
        if value != parameters[param].default:
            param_sig = parameters[param]
            if 'Metadata' in param_sig.qiime_type.name:
                assert False
            elif param_sig.qiime_type.name == 'Bool':
                cmd.append('--p-' + ('' if value else 'no-') + param.replace('_', '-'))
            else:
                cmd.append('--p-' + param.replace('_', '-'))
                cmd.append(str(value))
    required_artifacts = []
    cmd.append('--o-' + action['action']['output-name'].replace('_', '-'))
    cmd.append(results[uuid])
    for imput in action['action']['inputs']:
        (imput, uuid), = imput.items()
        if uuid is None:
            continue
        cmd.append('--i-' + imput.replace('_', '-'))
        cmd.append(results[uuid])
        required_artifacts.append(uuid)
    return ' '.join(cmd), required_artifacts

def get_commands(action, results, prov_dir, uuid=None):
    cmd, dependencies = get_command(action, results, prov_dir, uuid)
    commands = [cmd]
    for uuid in dependencies:
        with (prov_dir / 'artifacts' / uuid / 'action' / 'action.yaml').open() as fh:
            action = yaml.load(fh)
        commands.extend(get_commands(action, results, prov_dir, uuid))
    return commands

In [173]:
with (final_artifact._archiver.provenance_dir / 'action' / 'action.yaml').open() as fh:
    action = yaml.load(fh)        

In [174]:
commands = reversed(get_commands(action, results, final_artifact._archiver.provenance_dir, str(final_artifact.uuid)))
for cmd in OrderedDict([(c, None) for c in commands]):
    print(cmd)

qiime tools import --input-path 99_otu_taxonomy.txt --input-format HeaderlessTSVTaxonomyFormat --type FeatureData[Taxonomy] --output-path ref-tax.qza
qiime tools import --input-path 99_otus.fasta --input-format DNAFASTAFormat --type FeatureData[Sequence] --output-path ref-seq.qza
qiime feature-classifier extract-reads --p-f-primer GTGCCAGCMGCCGCGGTAA --p-r-primer GGACTACHVGGGTWTCTAAT --p-trunc-len 100 --o-reads ref-seq-515f-806r-100nt.qza --i-sequences ref-seq.qza
qiime feature-classifier fit-classifier-naive-bayes --o-classifier uniform.qza --i-reference-reads ref-seq-515f-806r-100nt.qza --i-reference-taxonomy ref-tax.qza
qiime tools import --input-path all.biom --input-format BIOMV210Format --type FeatureTable[Frequency] --output-path cheese.qza
qiime clawback sequence-variants-from-samples --o-sequences cheese-seq.qza --i-samples cheese.qza
qiime feature-classifier classify-sklearn --o-classification uniform-cheese.qza --i-reads cheese-seq.qza --i-classifier uniform.qza
qiime taxa c